In [9]:
import numpy as np
import pandas as pd

## IMPORT DATA

Data type should be made in a csv file in this order: 
   
ex:  
  
company_name,adv,dadv,average_salary(만원),total_sale(억원)  
   
삼성전자, 삼성 전자의 모든 장점 리뷰 데이터, 삼성 전자의 모든 단점 리뷰 데이터, 삼성전자의 평균 연봉 (만원), 삼성전자의 연 매출액 (억원)

In [10]:
df = pd.read_csv("sample_input.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df

,company_name,adv,dadv,average_salary,total_sale
0,현대카드,일단 오피스 환경이 쾌적한것은 장점대기업만의 성과급이나 복지의 혜택등이 좋다생각보다...,꼰대문화 부서에따라 존재. 굽신굽신 매일 퇴근은 정시에 포기 할일 다하면 다른업무...,8737,2300.0


In [11]:
r_title = df['company_name'].iloc[0] +"_result.txt"
f = open(r_title, "w")

## Topic Modeling

create topic modeling result

In [12]:
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from gensim.models import KeyedVectors

import pickle

model_adv = KeyedVectors.load("tp_model/tp_adv/tp_adv_model")
model_dadv = KeyedVectors.load("tp_model/tp_dadv/tp_dadv_model")

with open('input/corpus_adv.pkl', 'rb') as lf:
    corpus_adv = pickle.load(lf)
with open('input/corpus_dadv.pkl', 'rb') as lf:
    corpus_dadv = pickle.load(lf)

In [13]:
num_topics_adv = 6
num_topics_dadv = 7

output_df_adv = pd.DataFrame({'company_name':df.company_name})
for col in range(num_topics_adv):
    tmp = str(col)
    output_df_adv[tmp] = float(0)
output_df_adv.index = [i for i in range(len(output_df_adv))]

output_df_dadv = pd.DataFrame({'company_name':df.company_name})
for col in range(num_topics_dadv):
    tmp = str(col)
    output_df_dadv[tmp] = float(0)
output_df_dadv.index = [i for i in range(len(output_df_dadv))]

for i, dt in enumerate(model_adv.get_document_topics(corpus_adv)):
    for val in dt:

        idx = str(val[0])
        pt = val[1]

        output_df_adv.at[i,idx] = pt

for i, dt in enumerate(model_dadv.get_document_topics(corpus_dadv)):
    for val in dt:

        idx = str(val[0])
        pt = val[1]

        output_df_dadv.at[i,idx] = pt

In [14]:
# output_df

df = df.drop(['adv','dadv'], axis = 1)

# adv
output_df_adv = output_df_adv.dropna()

col_adv = list(output_df_adv.columns)

for i in range(1, len(col_adv)):
    col_adv[i] = "adv_topic_"+col_adv[i]
output_df_adv.columns = col_adv

# dadv
output_df_dadv = output_df_dadv.dropna()

col_dadv = list(output_df_dadv.columns)

for i in range(1, len(col_dadv)):
    col_dadv[i] = "dadv_topic_"+col_dadv[i]
output_df_dadv.columns = col_dadv

output_df = pd.merge(output_df_adv, output_df_dadv ,how='inner')
output_df = pd.merge(output_df, df ,how='inner')
output_df

,company_name,adv_topic_0,adv_topic_1,adv_topic_2,adv_topic_3,adv_topic_4,adv_topic_5,dadv_topic_0,dadv_topic_1,dadv_topic_2,dadv_topic_3,dadv_topic_4,dadv_topic_5,dadv_topic_6,average_salary,total_sale
0,현대카드,0.978254,0.0,0.01019,0.0,0.0,0.0,0.0,0.073691,0.0,0.013542,0.0,0.91249,0.0,8737,2300.0


In [15]:
# log transformation of output_df

for feature in list(output_df.columns)[1:-1]:
    if feature != 'dadv_topic_4':
        nonzero_indices = output_df[feature] != 0  # 0이 아닌 값의 인덱스를 찾음
        output_df[feature] = np.log1p(output_df[feature])
output_df = output_df.rename(columns={"average_salary":"average_salary(만원)", "total_sale":"total_sale(억원)"})
output_df

,company_name,adv_topic_0,adv_topic_1,adv_topic_2,adv_topic_3,adv_topic_4,adv_topic_5,dadv_topic_0,dadv_topic_1,dadv_topic_2,dadv_topic_3,dadv_topic_4,dadv_topic_5,dadv_topic_6,average_salary(만원),total_sale(억원)
0,현대카드,0.682214,0.0,0.010139,0.0,0.0,0.0,0.0,0.071102,0.0,0.013451,0.0,0.648406,0.0,9.075437,2300.0


In [16]:
f.write("Company name: " + str(output_df['company_name'][0])+'\n')
f.write('\n')
f.write('Topic Modeling Result: \n')
f.write('\n')

for i in output_df.columns[1:-2]:
    f.write(i+": "+str(output_df[i].iloc[0])+'\n')

run it into machine learning

In [17]:
import joblib
lgbm_model = joblib.load('ml_model/tp_lgbm_reg_model.pkl')

x_data = output_df.drop(['company_name'], axis = 1, inplace = False)

f.write('\n')
f.write("ML: \n")
f.write("\n")
f.write("Predicted turnover rate of the company using lgbm model is :"+ str(lgbm_model.predict(x_data))+'\n')

73

In [18]:
xgb_model = joblib.load('ml_model/tp_xgb_reg_model.pkl')

x_data = output_df.drop(['company_name'], axis = 1, inplace = False)

f.write("Predicted turnover rate of the company using xgb model is :" + str(xgb_model.predict(x_data)) +'\n')

72

In [19]:
tp_df = pd.read_csv('input/tp_df.csv')
tp_df = tp_df.drop(["Unnamed: 0","company_name","turn_over_rate","average_salary"], axis=1)

tmp = output_df.drop(["company_name","average_salary(만원)","total_sale(억원)"], axis=1)

summary = tp_df.describe()
summary = summary.drop(["count","std"],axis = 0)

In [20]:
categories = ["mean", "min","25%","50%","75%","max"]

topics = ['직업 안정성+휴식 (장점)','구내식당 + 제공서비스 만족 (장점)','여가 지원 (장점)','자율적 근무환경 (장점)','성과급 만족 (장점)','회사 시설 (장점)',
          '진급의 어려움 + 수직적 문화 (단점)','정규직으로의 전환 (단점)','평가체계부실 (단점)','근무지 불만 (단점)','낮은 발전 가능성(단점)',
          '고객 응대 + 성과에 대한 압박 (단점)','성과금 + 사내복지 부족 (단점)']
f.write('\n')
f.write('Featue Comparison: \n')
f.write('\n')
for i in range(len(summary.columns)):
    save = []
    # print(tmp.iloc[0,i])
    
    for j in range (len(summary.iloc[:,i])):
        if tmp.iloc[0,i] > summary.iloc[j,i]:
            save.append(categories[j])
    if len(save) >0:
        f.write("For topic: " + topics[i] +'\n')
        if save[0] == "mean":
            f.write("The topic modeling value is greater than mean and " + save[-1] + " of the data\n")
        else:
            f.write("The topic modeling value is greater than "+ save[-1]+" of the data\n")

In [21]:
f.close()